# Functions

In [1]:
def valueFilter(dataFrame,columnName,values):
    return dataFrame[~dataFrame[columnName].isin(values)]

# Formating Data

In [52]:
import pandas as pd
from scipy.stats import pearsonr
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV, train_test_split
import matplotlib.pyplot as plt
import statistics
from sklearn import linear_model
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score

# Import raw datasets

In [3]:
audi_raw = pd.read_csv('data/audi.csv')

In [4]:
bmw_raw = pd.read_csv('data/bmw.csv')

In [5]:
ford_raw = pd.read_csv('data/ford.csv')

In [6]:
hyundi_raw = pd.read_csv('data/hyundi.csv')

In [7]:
merc_raw = pd.read_csv('data/merc.csv')

In [8]:
skoda_raw = pd.read_csv('data/skoda.csv')

In [9]:
toyota_raw = pd.read_csv('data/toyota.csv')

In [10]:
vauxhall_raw = pd.read_csv('data/vauxhall.csv')

In [11]:
vw_raw = pd.read_csv('data/vw.csv')

# Merge into one dataframe

In [12]:
hyundi_raw = hyundi_raw.rename(columns={'tax(£)':'tax'})

In [13]:
df = [audi_raw, bmw_raw, ford_raw, hyundi_raw, merc_raw, skoda_raw, toyota_raw, vauxhall_raw, vw_raw]

In [58]:
cars = pd.concat(df)
cars['model'] = cars['model'].astype('category')
cars['transmission'] = cars['transmission'].astype('category')
cars['fuelType'] = cars['fuelType'].astype('category')
cars = pd.get_dummies(cars)


# Regression Model

In [62]:
cars_x = cars.drop('price', inplace = False, axis=1)
cars_y = cars['price']

In [63]:
cars_train, cars_test, price_train, price_test = train_test_split(cars_x, cars_y, test_size = 0.3, random_state=101)

In [64]:
mod = linear_model.LinearRegression()
mod.fit(cars_x, cars_y)

LinearRegression()

In [65]:
cars_pred = mod.predict(cars_test)

In [66]:
r2_score(price_test, cars_pred)

0.8579379076156233